In [1]:

import numpy as np
import altair as alt
import pandas as pd

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

import streamlit as st

In [9]:
host = st.secrets['snowflake']['host']
user = st.secrets['snowflake']['user']
password = st.secrets['snowflake']['password']
account = st.secrets['snowflake']['account']
warehouse = st.secrets['snowflake']['warehouse']
database = st.secrets['snowflake']['database']
schema = st.secrets['snowflake']['schema']

cd = {
    'host' : host,
    'user' : user,
    'password' : password,
    'account' : account,
    'warehouse' : warehouse,
    'database' : database,
    'schema' : schema
}

In [16]:
def snowflake_query(snowflake_schema, snowflake_table, cd):

    query = f"select * from {snowflake_schema}.{snowflake_table}"

    connector = snowflake.connector.connect(
        host=cd['host'],
        user=cd['user'],
        password=cd['password'],
        account=cd['account'],
        warehouse=cd['warehouse'],
        database=cd['database'],
        schema=cd['schema'])

    try:
        cursor = connector.cursor().execute(query)
    finally:
        connector.close()
    df = cursor.fetch_pandas_all()

    return df

In [19]:
raw_places = snowflake_query('RAW', 'CSV__PLACES_DATA', cd)
raw_transit = snowflake_query('RAW', 'CSV__TRANSIT_DATA', cd)
raw_index_meta = snowflake_query('RAW', 'JSON__INDEX_META', cd)
raw_property_details = snowflake_query('RAW', 'JSON__DETAILS', cd)

2022-12-10 14:24:10.547 Snowflake Connector for Python Version: 2.7.12, Python Version: 3.9.13, Platform: macOS-10.16-x86_64-i386-64bit
2022-12-10 14:24:10.550 This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2022-12-10 14:24:12.042 query: [select * from RAW.CSV__PLACES_DATA]
2022-12-10 14:24:12.620 query execution done
2022-12-10 14:24:12.622 closed
2022-12-10 14:24:12.881 No async queries seem to be running, deleting session
2022-12-10 14:24:13.141 Failed to add log to telemetry.
Traceback (most recent call last):
  File "/Users/tanhtra/opt/anaconda3/envs/bootcamp/lib/python3.9/site-packages/snowflake/connector/telemetry.py", line 96, in try_add_log_to_batch
    self.add_log_to_batch(telemetry_data)
  File "/Users/tanhtra/opt/anaconda3/envs/bootcamp/lib/python3.9/site-packages/snowflake/connector/

In [21]:
stg_details = snowflake_query('STAGING', 'STG__DETAILS', cd)
stg_developers = snowflake_query('STAGING', 'STG__DEVELOPERS', cd)
stg_features = snowflake_query('STAGING', 'STG__FEATURES', cd)
stg_places = snowflake_query('STAGING', 'STG__PLACES', cd)
stg_projects = snowflake_query('STAGING', 'STG__PROJECTS_LIST', cd)
stg_transit = snowflake_query('STAGING', 'STG__TRANSIT', cd)
stg_vacancy = snowflake_query('STAGING', 'STG__VACANCY', cd)

2022-12-10 14:28:39.783 Snowflake Connector for Python Version: 2.7.12, Python Version: 3.9.13, Platform: macOS-10.16-x86_64-i386-64bit
2022-12-10 14:28:39.788 This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2022-12-10 14:28:41.160 query: [select * from STAGING.STG__DETAILS]
2022-12-10 14:28:42.472 query execution done
2022-12-10 14:28:42.473 closed
2022-12-10 14:28:42.688 No async queries seem to be running, deleting session
2022-12-10 14:28:42.945 Failed to add log to telemetry.
Traceback (most recent call last):
  File "/Users/tanhtra/opt/anaconda3/envs/bootcamp/lib/python3.9/site-packages/snowflake/connector/telemetry.py", line 96, in try_add_log_to_batch
    self.add_log_to_batch(telemetry_data)
  File "/Users/tanhtra/opt/anaconda3/envs/bootcamp/lib/python3.9/site-packages/snowflake/connector/